<a href="https://colab.research.google.com/github/YanaKhlusova/ML_course/blob/main/Khlusova_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Генерация заголовков для статей (по данным arXiv)

Вечная проблема -- как придумать креативное название для статьи. Вот сейчас и погенерим...

In [1]:
import re
import numpy as np
import pandas as pd
import kagglehub
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# скачаем датасет с абстрактами и тайтлами статей на архиве с kaggle
# изначально я хотела генерить абстракты, но есть ощущение, что это будет слишком тяжеловесно
path = kagglehub.dataset_download("spsayakpaul/arxiv-paper-abstracts")

df = pd.read_csv(path + '/arxiv_data.csv')

df.head()

100%|██████████| 44.6M/44.6M [00:00<00:00, 270MB/s]

Extracting files...


,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [3]:
df['titles'][:5].to_list()

['Survey on Semantic Stereo Matching / Semantic Depth Estimation',
 'FUTURE-AI: Guiding Principles and Consensus Recommendations for Trustworthy Artificial Intelligence in Future Medical Imaging',
 'Enforcing Mutual Consistency of Hard Regions for Semi-supervised Medical Image Segmentation',
 'Parameter Decoupling Strategy for Semi-supervised 3D Left Atrium Segmentation',
 'Background-Foreground Segmentation for Interior Sensing in Automotive Industry']

## Препроцессинг

Приведу текст к нижнему регистру (для снижения размерности данных), на всякий случай прогоню удаление красных строк и экстрапробелов -- вдруг появилось зашумление после OCR. Знаки пунктуации оставляю, их мало, и они типичны для статей (двоеточие и запятые). Стоп-слова тоже важны для связности.

In [4]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [5]:
df['cleaned_titles'] = df['titles'].apply(preprocess_text)

In [7]:
data = df['cleaned_titles'][:1000]
# чтоб колаб не умер

In [8]:
# Инициализируем токенизатор
tokenizer = Tokenizer()

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(data)

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(data)

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

In [9]:
# Преобразуем списки в массивы numpy
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)

# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [10]:
# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Обучаем модель
history = model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.0623 - loss: 6.7118 - val_accuracy: 0.0707 - val_loss: 6.0381
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0792 - loss: 5.6620 - val_accuracy: 0.0968 - val_loss: 6.0504
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1199 - loss: 5.5103 - val_accuracy: 0.1154 - val_loss: 6.0466
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1483 - loss: 5.3514 - val_accuracy: 0.1302 - val_loss: 6.0337
Epoch 5/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.1754 - loss: 5.1674 - val_accuracy: 0.1435 - val_loss: 5.9787
Epoch 6/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.1870 - loss: 5.0073 - val_accuracy: 0.1483 - val_loss: 5.9200
Epoch 7/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2034 - loss: 4.7919 - val_accuracy: 0.1489 - val_loss: 5.8877
Epoch 8/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2107 - loss: 4.6296 - val_accuracy: 

Экьюраси неплохая получилась, но тут у нас и данные должны быть +- сбалансированны по теме (след., использовать похожие слова), они короткие и их мало.

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 24, 100)        │       211,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2112)           │       318,912 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,042,138 (7.79 MB)

 Trainable params: 680,712 (2.60 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,361,426 (5.19 MB)

In [13]:
# Функция для генерации текста
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [14]:
# Генерируем новый заголовок
generated_text = generate_text("Graph", 5, X.shape[1])
print(generated_text)
# куда же без графов)
# пяти слов явно не хватает

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Graph cut segmentation using deep convolutional


In [15]:
generated_text = generate_text("Graph", 7, X.shape[1])
print(generated_text)
# звучит по крайней мере связно, если убрать cut

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Graph cut segmentation using deep convolutional neural network


In [16]:
# здесь я вспомнила, что у меня lowercase
generated_text = generate_text("graph", 7, X.shape[1])
print(generated_text)
# на генерацию никак не повлияло, что хорошо

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
graph cut segmentation using deep convolutional neural network


In [17]:
generated_text = generate_text("rag", 7, X.shape[1])
print(generated_text)
# раг, наверное, не попал в выборку, из-за чего модель стала использовать слова,
# имеющие похожее кол-во символов и, по-видимому, частоту испольозования в тексте (= редкие слова)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
rag an mrf unet product of experts for


In [18]:
# мб это есть в датасете
generated_text = generate_text("semantic web", 5, X.shape[1])
print(generated_text)
# видимо, нет

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
semantic web a new perspective on medical


In [19]:
generated_text = generate_text("logic", 7, X.shape[1])
print(generated_text)
# лоджик, видимо, тоже нет, либо редкое слово

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
logic an mrf unet product of experts for


In [20]:
generated_text = generate_text("neural network", 7, X.shape[1])
print(generated_text)
# интересно, что же там дальше

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
neural network construction for interactive image segmentation using persistent


In [22]:
generated_text = generate_text("neural network", 11, X.shape[1])
print(generated_text)
# если убрать for будет связно

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
neural network construction for interactive image segmentation using persistent homology data points for


In [23]:
generated_text = generate_text("neural networks", 11, X.shape[1])
print(generated_text)
# идеально)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
neural networks with deep learning for visual explanation and statistical estimations of image


In [24]:
generated_text = generate_text("deep learning", 10, X.shape[1])
print(generated_text)
# оптимал стоппинг проблемс это, конечно, хорошо
# как заголовок звучит связно

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
deep learning for ranking response surfaces with applications to optimal stopping problems


In [25]:
generated_text = generate_text("ranking", 11, X.shape[1])
print(generated_text)
# ух-ты, это мультилингвальный датасет :D

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
ranking de imagens utilizando competição e cooperação entre partículas partículas partículas for


**Итог:**

модель обучилась быстро, неплохо, и показала удовлетворительные результаты генерации с точки зрения связности текста (= названия звучат хорошо), если слово(сочетание) было в обучающих данных. Но стоит помнить, что я взяла короткие тексты и маленький кусочек данных. Кроме того, пунктуация в названиях так и не появилась, хотя сейчас многие статьи имеют вид <название продукта: описание продукта>. Смысловую составляющую названий я не оценивала из-за недостатка знаний в области (не могу сказать, нет ли фактических ошибок.)

Гипотеза: если слова не было в обучающих данных или оно было редким, модель подбирает наиболее подходящие по структуре/частотности слова. (Гипотеза не проверена).

In [26]:
# Сохраняем модель
model.save('arxiv_papers_title_generator.keras')